# Benchmarks

## Packages & functions

In [44]:
k_folds = 10

In [ ]:
import sys
from tqdm import tqdm
import dill
from copy import deepcopy
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, roc_auc_score, auc, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def compute_pr_auc(y_true, y_score):
    precision, recall, thresholds = precision_recall_curve(y_true, y_score)
    auc_value = auc(recall, precision)
    return auc_value

def eval(factor_list, classes, kfolds_no=k_folds):

    roc_auc = np.zeros((len(factor_list), kfolds_no))
    pr_auc = np.zeros((len(factor_list), kfolds_no))
    f1 = np.zeros((len(factor_list), kfolds_no))

    np.random.seed(123)
    cv = StratifiedKFold(kfolds_no, random_state=123, shuffle=True)  

    for i in range(len(factor_list)):

        X = factor_list[i]
        y = classes
        roc_auc_tmp = []
        pr_auc_tmp = []
        f1_tmp = []
        i0 = 0
        for train_index, test_index in cv.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            i0 += 1

            mod = RandomForestClassifier(n_estimators=100, random_state=123+i0)
            mod.fit(X_train, y_train)
            y_prob = mod.predict_proba(X_test)[:,1]
            roc_auc_tmp.append(roc_auc_score(y_test, y_prob))
            pr_auc_tmp.append(compute_pr_auc(y_test, y_prob))
            f1_tmp.append(f1_score(y_test, y_prob > 0.5))

        roc_auc[i,:] = roc_auc_tmp
        pr_auc[i,:] = pr_auc_tmp
        f1[i,:] = f1_tmp


    return roc_auc, pr_auc, f1

In [ ]:
def eval_average(factor_list, list_of_names, classes, kfolds_no=k_folds):

    classes_unique = np.unique(classes, return_counts=True)

    classes_names = np.sort(classes_unique[0])

    roc_auc = np.zeros((len(factor_list), kfolds_no))
    pr_auc = np.zeros((len(factor_list), kfolds_no))
    f1 = np.zeros((len(factor_list), kfolds_no))

    roc_auc_all = np.zeros((classes_names.shape[0], len(factor_list), kfolds_no))
    pr_auc_all = np.zeros((classes_names.shape[0], len(factor_list), kfolds_no))
    f1_all = np.zeros((classes_names.shape[0], len(factor_list), kfolds_no))

    df_all = pd.DataFrame()

    for i in range(classes_names.shape[0]):
        class_tmp = np.zeros(classes.shape[0])
        class_tmp[classes == classes_names[i]] = 1
        roc_auc_tmp, pr_auc_tmp, f1_tmp = eval(factor_list, class_tmp, kfolds_no)

        for measure in ['roc_auc', 'pr_auc', 'f1']:
            if measure == 'roc_auc':
                perf = roc_auc_tmp
            if measure == 'pr_auc':
                perf = pr_auc_tmp
            if measure == 'f1':
                perf = f1_tmp
            res_tmp = pd.DataFrame(perf.T)
            res_tmp.columns = list_of_names
            res_tmp = pd.melt(res_tmp, var_name='model', value_name='value')
            res_tmp['measure'] = measure
            res_tmp['class'] = classes_names[i]
            df_all = pd.concat([df_all, res_tmp])
            

        roc_auc_all[i,:,:] = roc_auc_tmp
        pr_auc_all[i,:,:] = pr_auc_tmp
        f1_all[i,:,:] = f1_tmp

        roc_auc += classes_unique[1][i]*roc_auc_tmp
        pr_auc += classes_unique[1][i]*pr_auc_tmp
        f1 += classes_unique[1][i]*f1_tmp

    roc_auc = roc_auc/np.sum(classes_unique[1])
    pr_auc = pr_auc/np.sum(classes_unique[1])
    f1 = f1/np.sum(classes_unique[1])

    return roc_auc, pr_auc, f1, df_all

In [ ]:
def eval_comparison_df(list_of_factors, list_of_names, classes):

    classes_unique = np.unique(classes, return_counts=True)

    classes_names = np.sort(classes_unique[0])

    df_all = pd.DataFrame()

    if classes_names.shape[0] == 2:
        roc_auc, pr_auc, f1 = eval(list_of_factors, classes)
    if classes_names.shape[0] > 2:
        roc_auc, pr_auc, f1, df_all = eval_average(list_of_factors, list_of_names, classes)
    
    res = pd.DataFrame()

    for measure in ['roc_auc', 'pr_auc', 'f1']:
        if measure == 'roc_auc':
            perf = roc_auc
        if measure == 'pr_auc':
            perf = pr_auc
        if measure == 'f1':
            perf = f1
        res_tmp = pd.DataFrame(perf.T)
        res_tmp.columns = list_of_names
        res_tmp = pd.melt(res_tmp, var_name='model', value_name='value')
        res_tmp['measure'] = measure
        res = pd.concat([res, res_tmp])

    return res, df_all

In [50]:
def get_factors(file_list):
    factor_list = []
    for i in range(len(file_list)):
        with open(file_list[i], "rb") as input_file:
            factors, _, _ = dill.load(input_file)
        factor_list.append(factors)
    return factor_list

In [ ]:
def get_structered_representation(file_list, index_lda_log, index_lda=None):
    factor_list = []
    for i in range(len(file_list)):
        with open(file_list[i], "rb") as input_file:
            res = dill.load(input_file)
        if i != index_lda_log and i != index_lda:
            rep = res[3]
        if i == index_lda_log:
            rep = np.log(res[1])
        if i == index_lda:
            rep = res[1]
        factor_list.append(rep)
    return factor_list

## MOSEI

In [57]:
with open('data_transformed/mosei_preprocessed_data_sentence1words.pkl', "rb") as input_file:
    factm_input = dill.load(input_file)

In [ ]:
classes = (factm_input['labels'] > 0) + 0.0

In [ ]:
file_list = ['factm_fa_mosei.pkl',
             'fa_ctm_mosei.pkl',
             'mofa_mosei.pkl',
             'muvi_mosei.pkl']
model_list = ['factm', 'fa_ctm', 'mofa', 'muvi']
factor_list = get_factors(file_list)
results, _ = eval_comparison_df(factor_list, model_list, classes)
results['dataset'] = 'mosei'
# results.to_csv('mosei_factors_classification.csv')

In [ ]:
file_list = ['factm_ctm_mosei.pkl',
              'ctm_mosei.pkl',
              'lda_mosei.pkl']
model_list = ['factm', 'ctm', 'lda']

structered_representation_list = get_structered_representation(file_list, 2)
results, _ = eval_comparison_df(structered_representation_list, model_list, classes)
results['dataset'] = 'mosei'
# results.to_csv('mosei_structured_classification.csv')

## MOSI

In [42]:
with open('data_transformed/mosi_preprocessed_data_sentence1words.pkl', "rb") as input_file:
    factm_input = dill.load(input_file)

In [23]:
classes = (factm_input['labels'] > 0) + 0.0

In [ ]:
file_list = ['factm_fa_mosi.pkl',
             'fa_ctm_mosi.pkl',
             'mofa_mosi.pkl',
             'muvi_mosi.pkl']
model_list = ['factm', 'fa_ctm', 'mofa', 'muvi']
factor_list = get_factors(file_list)
results, _ = eval_comparison_df(factor_list, model_list, classes)
results['dataset'] = 'mosi'
# results.to_csv('mosi_factors_classification.csv')

In [ ]:
file_list = ['factm_ctm_mosi.pkl',
              'ctm_mosi.pkl',
              'lda_mosi.pkl']
model_list = ['factm', 'ctm', 'lda']

structered_representation_list = get_structered_representation(file_list, 2)
results, _ = eval_comparison_df(structered_representation_list, model_list, classes)
results['dataset'] = 'mosi'
# results.to_csv('mosi_structured_classification.csv')

## MIREX

In [ ]:
with open('mirex_info.pkl', "rb") as input_file:
    [feature_names, classes, categories, vocab] = dill.load(input_file)

In [ ]:
file_list = ['factm_fa_mirex.pkl',
             'fa_ctm_mirex.pkl',
             'mofa_mirex.pkl',
             'muvi_mirex.pkl']
model_list = ['factm', 'fa_ctm', 'mofa', 'muvi']
factor_list = get_factors(file_list)
results, results2 = eval_comparison_df(factor_list, model_list, classes)
results2['dataset'] = 'mirex'
# results2.to_csv('mirex_factors_classification.csv')

In [ ]:
file_list = ['factm_ctm_mirex.pkl',
              'ctm_mirex.pkl',
              'lda_mirex.pkl']
model_list = ['factm', 'ctm', 'lda']

structered_representation_list = get_structered_representation(file_list, 2)
results, results2 = eval_comparison_df(structered_representation_list, model_list, classes)
results2['dataset'] = 'mirex'
# results2.to_csv('mirex_structured_classification.csv')